# Experimento 2

Jeffrey Leiva Cascante 2021016720

Richard León Chinchilla 2019003759


In [ ]:
import sys
import os
import hydra
from config.config import Config
from DenoisingAutoencoder import DenoisingAutoencoder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import torch_lightning as LookupError

notebook_dir = os.path.dirname(os.path.abspath(os.getcwd()))
config_dir = os.path.join(notebook_dir, 'proyecto-transfer-learning\\Config')

@hydra.main(config_path=config_dir, config_name="config")
    def main(cfg: Config):
        